In [3]:
import pandas as pd
import numpy as np
import datetime
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
import io
import base64
from matplotlib import pyplot as plt

# ==============================================================================
# 1. データ読み込みと初期準備
# ==============================================================================

# --- データファイルのパス設定 ---
# 🚨 Colabで実行する場合: これらのCSVファイルは、このノートブックと同じディレクトリに配置してください。
# 🚨 必要に応じて、ファイルのパスを適宜修正してください。
train_path = '/content/train.csv'          # 訓練データ（過去の販売実績）
calendar_path = '/content/2019_calendar.csv' # カレンダーデータ（休日情報など）
bouquet_master_path = '/content/bouquet_master.csv' # 花束の構成データ（花材と本数）

# 訓練データとカレンダーデータを読み込み
train_df = pd.read_csv(train_path)
calendar_df = pd.read_csv(calendar_path)

# 花束マスターデータをファイルパスから読み込み
try:
    bouquet_master_df = pd.read_csv(bouquet_master_path)
    # カラム名を統一（訓練データとマージするため）
    bouquet_master_df.rename(columns={'商品名': '商品の種類'}, inplace=True)
except FileNotFoundError:
    print(f"⚠️ エラー: {bouquet_master_path} が見つかりません。代替データを使用します。")
    # ファイルが見つからない場合の代替データ（ハードコーディング）
    bouquet_master_data = """商品名,バラ,ガーベラ,カーネーション,本数合計
花束A,1,1,1,3
花束B,2,1,1,4
花束C,1,2,1,4
花束D,1,1,2,4
花束E,2,2,1,5
花束F,2,1,2,5
花束G,1,2,2,5
花束H,2,2,2,6
"""
    bouquet_master_df = pd.read_csv(io.StringIO(bouquet_master_data))
    bouquet_master_df.rename(columns={'商品名': '商品の種類'}, inplace=True)


# 日付型への変換（特徴量エンジニアリングのため必須）
train_df['日付'] = pd.to_datetime(train_df['日付'])
calendar_df['日付'] = pd.to_datetime(calendar_df['日付'])

# --- 天気特徴量のシミュレーションと準備 ---
# 訓練データに天気がないため、予測モデルの学習用にランダムな天気を割り当てる
np.random.seed(42)
weather_options_for_ohe = ['晴れ', '曇り', '雨']
random_weather = np.random.choice(weather_options_for_ohe, size=len(train_df['日付'].unique()))
temp_weather_df = pd.DataFrame({
    '日付': train_df['日付'].unique(),
    '天気': random_weather
})
# 訓練データにシミュレートした天気データをマージ
train_df = pd.merge(train_df, temp_weather_df, on='日付', how='left')

# --- カテゴリ特徴量（天気、商品）のOne-Hotエンコーダーの定義と準備 ---
# モデルがカテゴリ変数（文字情報）を扱えるように数値に変換する
weather_ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# OHEは '晴れ', '曇り', '雨' のみを学習。予測時に「不明」が入ると全て0として処理される。
weather_ohe.fit(np.array(weather_options_for_ohe).reshape(-1, 1))

product_options = train_df['商品の種類'].unique().tolist()
product_ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
product_ohe.fit(np.array(product_options).reshape(-1, 1))


# ==============================================================================
# 2. 特徴量エンジニアリング関数
# ==============================================================================

def apply_weather_encoding(df, ohe):
    """データフレームに天気のOne-Hotエンコーディングを適用する"""
    if '天気' in df.columns:
        df['天気'] = df['天気'].astype(str)
        encoded_data = ohe.transform(df['天気'].values.reshape(-1, 1))
        encoded_df = pd.DataFrame(encoded_data, columns=ohe.get_feature_names_out(['天気']), index=df.index)
        df = pd.concat([df.drop('天気', axis=1), encoded_df], axis=1)

    # OHEが学習したすべてのカラム（例: '天気_晴れ'）がデータフレームに存在することを確認し、なければ0で埋める
    for col_name in ohe.get_feature_names_out(['天気']):
        if col_name not in df.columns:
            df[col_name] = 0.0
    return df

def apply_product_encoding(df, ohe):
    """データフレームに商品名のOne-Hotエンコーディングを適用する"""
    if '商品の種類' in df.columns:
        df['商品の種類'] = df['商品の種類'].astype(str)
        encoded_data = ohe.transform(df['商品の種類'].values.reshape(-1, 1))
        encoded_df = pd.DataFrame(encoded_data, columns=ohe.get_feature_names_out(['商品']), index=df.index)
        df = pd.concat([df.drop('商品の種類', axis=1), encoded_df], axis=1)

    # OHEが学習したすべてのカラムがデータフレームに存在することを確認
    for col_name in ohe.get_feature_names_out(['商品']):
        if col_name not in df.columns:
            df[col_name] = 0.0
    return df

def engineer_date_and_calendar_features(df, calendar_df):
    """
    日付から季節性や曜日などの特徴量を抽出し、カレンダー情報と結合する。
    """
    df['日付'] = pd.to_datetime(df['日付'])
    df['month'] = df['日付'].dt.month          # 月
    df['day'] = df['日付'].dt.day              # 日
    df['day_of_week_int'] = df['日付'].dt.dayofweek # 曜日 (月曜=0, 日曜=6)
    df['day_of_year'] = df['日付'].dt.dayofyear  # 年間通算日
    df['quarter'] = df['日付'].dt.quarter      # 四半期

    # カレンダー情報をマージして休日フラグを作成
    df_merged = pd.merge(df, calendar_df, on='日付', how='left')
    df_merged['is_holiday'] = df_merged['休日'].fillna(0).astype(int) # 休日を0/1フラグに

    return df_merged

def create_lagged_sales_features(df, lag_days=[7, 365]):
    """
    全商品の合計販売量に基づいたラグ特徴量（過去の売上実績）を作成する。
    """
    # 日付ごとの合計販売量を計算
    all_sales = df.groupby('日付')['販売量'].sum().reset_index()
    all_sales.rename(columns={'販売量': 'total_sales'}, inplace=True)
    all_sales = all_sales.set_index('日付')

    lagged_df = all_sales[['total_sales']].copy()
    # 指定された日数分シフト（ラグ）して特徴量を作成
    for lag in lag_days:
        lagged_df[f'total_sales_lag_{lag}'] = lagged_df['total_sales'].shift(lag)

    return lagged_df.reset_index().drop(columns=['total_sales'])


# ==============================================================================
# 3. モデルの訓練
# ==============================================================================

# 特徴量エンジニアリングを実行
train_engineered_df = engineer_date_and_calendar_features(train_df.copy(), calendar_df.copy())
lagged_train_features = create_lagged_sales_features(train_df.copy())
train_engineered_df = pd.merge(train_engineered_df, lagged_train_features, on='日付', how='left')

# カテゴリ特徴量をOne-Hotエンコーディング
train_engineered_df = apply_weather_encoding(train_engineered_df, weather_ohe)
train_engineered_df = apply_product_encoding(train_engineered_df, product_ohe)

# ラグ特徴量の欠損値を0で補完（訓練データの最初期には過去データがないため）
for lag in [7, 365]:
    train_engineered_df[f'total_sales_lag_{lag}'] = train_engineered_df[f'total_sales_lag_{lag}'].fillna(0)

train_all_products = train_engineered_df.copy()

# モデルに学習させる特徴量のリストを定義
numerical_weather_cols = ['平均気温(℃)', '最高気温(℃)', '最低気温(℃)', '平均風速(m/s)', '最大風速(m/s)', '最大瞬間風速(m/s)']
features = [
    'month', 'day', 'day_of_week_int', 'day_of_year', 'quarter', '給料日',
    'is_holiday', 'total_sales_lag_7', 'total_sales_lag_365'
] + list(weather_ohe.get_feature_names_out(['天気'])) \
  + list(product_ohe.get_feature_names_out(['商品'])) \
  + numerical_weather_cols

# 特徴量(X)と目的変数(y)を定義
X = train_all_products[features]
y = train_all_products['販売量']

# 欠損値補完用の平均値を計算（主に気象データ。予測時にも利用する）
X_train_mean_imputation_values = X[numerical_weather_cols].mean().to_dict()
for col, mean_val in X_train_mean_imputation_values.items():
    if col in X.columns:
        X[col] = X[col].fillna(mean_val) # 訓練データ内の気象データの欠損値を平均で補完

X_train_columns = X.columns.tolist()

# ランダムフォレストモデルを訓練
model = RandomForestRegressor(random_state=42, n_estimators=100)
model.fit(X, y)


# ==============================================================================
# 4. 予測、花材計算、説明生成関数
# ==============================================================================

def calculate_flower_counts(product_name, predicted_sales_volume, master_df):
    """予測された販売量に基づき、マスターデータから各花材の必要本数を計算する。"""
    composition = master_df[master_df['商品の種類'] == product_name]
    if composition.empty:
        return {}

    estimated_flower_counts = {}
    flower_cols = ['バラ', 'ガーベラ', 'カーネーション']

    for flower in flower_cols:
        count_per_bouquet = composition[flower].iloc[0] # 花束あたりの本数
        total_count = predicted_sales_volume * count_per_bouquet
        estimated_flower_counts[flower] = int(round(total_count))
    return estimated_flower_counts

def generate_sophisticated_explanation(predicted_sales, product_name, engineered_features, model, event_name=None, selected_weather=None):
    """モデルの特徴量重要度に基づいて、予測の根拠を日本語で説明する。"""
    explanation = f"\n**商品（{product_name}）の予測根拠:**\n"
    explanation += f"予測売上: **{predicted_sales:.0f} 本**\n"

    if selected_weather and selected_weather != '不明':
        explanation += f"- **天気予報**: {selected_weather}の影響を考慮\n"
    else:
        explanation += "- **天気予報**: 情報がないため、気象データの過去平均値で予測\n"

    if event_name:
        explanation += f"- **イベント**: {event_name}の影響を考慮\n"

    # ラグ特徴量の重要度を表示（簡略化）
    if hasattr(model, 'feature_importances_'):

        # 過去7日間の総販売量（販売トレンド）
        lag_7_val = engineered_features['total_sales_lag_7'].iloc[0]
        explanation += f"- **過去7日間の総販売量**: {lag_7_val:.0f} 本（トレンド要因）\n"

        # 曜日
        weekday_map = {0: '月', 1: '火', 2: '水', 3: '木', 4: '金', 5: '土', 6: '日'}
        day_name = weekday_map.get(engineered_features['day_of_week_int'].iloc[0], '特定の日')
        explanation += f"- **曜日**: {day_name}曜日の販売傾向が強く影響\n"

    return explanation

def predict_and_display_results(button):
    """
    ユーザーの入力に基づいて、選択されたすべての商品の販売量を予測し、結果をまとめて表示するメインロジック。
    """
    # 修正点: 予測前に前回までの出力をクリアする
    with output_area:
        clear_output(wait=True) # IPython.display から clear_output を使用

        input_date_obj = date_input.value
        input_date = datetime.datetime.combine(input_date_obj, datetime.datetime.min.time())
        selected_weather = weather_dropdown.value
        user_selected_products = product_select_multiple.value

        if not user_selected_products:
            display(Markdown("⚠️ **エラー**: 予測する商品を1つ以上選択してください。"))
            return

        # 予測対象の確定：「すべて」が選択されているかチェック
        if 'すべて' in user_selected_products:
            # マスターデータ内のすべての花束を対象とする
            selected_products = bouquet_master_df['商品の種類'].unique().tolist()
            selection_header = " (全商品対象)"
        else:
            selected_products = user_selected_products
            selection_header = ""

        # イベント情報の特定（全商品共通）
        event_name = None
        events = {(2, 14): 'バレンタインデー', (12, 24): 'クリスマスイブ', (12, 25): 'クリスマス'}
        event_name = events.get((input_date.month, input_date.day))
        if input_date.month == 5 and input_date.weekday() == 6:
            # 母の日（5月第2日曜日）の判定ロジック
            first_day_of_month = datetime.date(input_date.year, input_date.month, 1)
            first_day_of_month_weekday = first_day_of_month.weekday()
            first_sunday = 7 - first_day_of_month_weekday if first_day_of_month_weekday != 6 else 1
            second_sunday = first_sunday + 7
            if input_date.day == second_sunday:
                event_name = '母の日'

        results = []

        # --- ラグ特徴量の共通計算 ---
        # 過去の総販売量は、予測対象日に関わらず共通で計算できる
        temp_input_df = pd.DataFrame({'日付': [input_date], '天気': [selected_weather], '商品の種類': [selected_products[0]], '販売量': [0]})
        combined_df_for_lag = pd.concat([train_df.copy(), temp_input_df], ignore_index=True)
        lagged_test_features = create_lagged_sales_features(combined_df_for_lag)

        # 7日前と365日前の総販売量を取得。データがない場合は訓練データの平均値で補完
        lagged_7_val = lagged_test_features[lagged_test_features['日付'] == input_date - datetime.timedelta(days=7)]['total_sales_lag_7'].values
        lagged_365_val = lagged_test_features[lagged_test_features['日付'] == input_date - datetime.timedelta(days=365)]['total_sales_lag_365'].values
        lag_7 = lagged_7_val[0] if lagged_7_val.size > 0 else X_train_mean_imputation_values.get('total_sales_lag_7', 0.0)
        lag_365 = lagged_365_val[0] if lagged_365_val.size > 0 else X_train_mean_imputation_values.get('total_sales_lag_365', 0.0)

        # 総合花材必要量の集計を初期化
        total_flower_needs = {'バラ': 0, 'ガーベラ': 0, 'カーネーション': 0}

        # --- 選択されたすべての商品に対して予測をループ実行 ---
        for product_name in selected_products:
            # 予測用データフレームを作成
            input_df = pd.DataFrame({'日付': [input_date], '天気': [selected_weather], '商品の種類': [product_name], '販売量': [0]})

            # 特徴量エンジニアリングとエンコーディング
            engineered_features_for_prediction = engineer_date_and_calendar_features(input_df.copy(), calendar_df.copy())
            engineered_features_for_prediction = apply_weather_encoding(engineered_features_for_prediction, weather_ohe)
            engineered_features_for_prediction = apply_product_encoding(engineered_features_for_prediction, product_ohe)

            # ラグ特徴量を設定
            engineered_features_for_prediction['total_sales_lag_7'] = lag_7
            engineered_features_for_prediction['total_sales_lag_365'] = lag_365

            # 訓練時と同じ特徴量セットを抽出
            X_input = engineered_features_for_prediction[X_train_columns].copy()

            # 欠損値補完（数値の気象データは訓練データ平均値で補完する）
            for col, mean_val in X_train_mean_imputation_values.items():
                if col in X_input.columns:
                    X_input[col] = X_input[col].fillna(mean_val)
                else:
                    X_input[col] = mean_val

            # 予測実行
            predicted_sales = model.predict(X_input)[0]
            predicted_sales = max(0, predicted_sales) # マイナス予測を0に修正

            # 必要花材数の計算と総計への集計
            estimated_flower_counts = calculate_flower_counts(product_name, predicted_sales, bouquet_master_df)
            for flower, count in estimated_flower_counts.items():
                total_flower_needs[flower] += count

            # 結果を格納
            results.append({
                'product_name': product_name,
                'predicted_sales': predicted_sales,
                'X_input': X_input,
                'event_name': event_name,
                'selected_weather': selected_weather
            })

        # --- 総合結果の表示 ---

        display(Markdown(f"\n# 🌸 花束の売上予測結果 ({input_date.strftime('%Y年%m月%d日')}){selection_header}"))

        # 総合花材必要量の表示
        display(Markdown(f"## 🧺 選択したすべての花束に必要な花材の総計 (**重要: この本数を仕入れてください**)"))
        total_output = ""
        for flower, total_count in total_flower_needs.items():
            total_output += f"- **{flower}**: **{total_count} 本**\n"
        display(Markdown(total_output))

        # 個別予測結果の表示
        display(Markdown("\n---"))
        display(Markdown("## 🎯 個別花束の予測売上本数と根拠"))

        for result in results:
            product_name = result['product_name']
            sales = result['predicted_sales']

            display(Markdown(f"### ➡️ {product_name} の予測売上: **{sales:.0f} 本**"))

            # 予測の根拠表示
            explanation = generate_sophisticated_explanation(sales, product_name, result['X_input'], model, result['event_name'], result['selected_weather'])
            print(explanation)
            print("---") # 区切り線



# ==============================================================================
# 5. ユーザーインターフェース（UI）の定義と表示
# ==============================================================================

# 日付選択ウィジェット
date_input = widgets.DatePicker(
    description='予測したい日付:',
    disabled=False,
    value=datetime.date.today()
)

# 天気予報の選択肢に「不明」を使用
weather_options_with_unknown = ['不明'] + weather_options_for_ohe # ['不明', '晴れ', '曇り', '雨']
weather_dropdown = widgets.Dropdown(
    options=weather_options_with_unknown,
    value='不明',
    description='天気予報 (任意):',
    disabled=False,
)

# 商品選択肢のリストの先頭に「すべて」を追加
all_product_options = ['すべて'] + product_options
product_select_multiple = widgets.SelectMultiple(
    options=all_product_options,
    value=['すべて'],
    description='商品名 (複数可):',
    disabled=False,
    rows=8
)

# 予測実行ボタン
predict_button = widgets.Button(description="売上を予測する")
# ボタンクリック時のイベントハンドラを設定
predict_button.on_click(predict_and_display_results)

# 予測結果を表示するための Output ウィジェットを定義
output_area = widgets.Output()

# 入力ウィジェットをグループ化
inputs = widgets.VBox([
    widgets.Label(value="--- 予測したい商品と日付を選択してください ---"),
    product_select_multiple,
    date_input,
    widgets.Label(value="--- 天気予報の選択（任意） ---"),
    weather_dropdown,
])

# ユーザーへの指示とUIの表示
print("📅 予測対象の商品（「すべて」を含む複数選択可）と日付、天気予報を入力してからボタンをクリックしてください:")
# UIと出力エリアをまとめて表示
display(widgets.VBox([inputs, predict_button, output_area]))

/tmp/ipython-input-2740820332.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].fillna(mean_val) # 訓練データ内の気象データの欠損値を平均で補完


📅 予測対象の商品（「すべて」を含む複数選択可）と日付、天気予報を入力してからボタンをクリックしてください:
